In [1]:
import re
import json
from io import BytesIO

import requests
import pandas as pd
from glob import glob
from PIL import Image

In [2]:
ad_keywords = [
    "내돈내산",
    "지원을받아",
    "지원받아",
    "후원을받아",
    "후원받아",
    "원고료",
    "체험단",
    "제공받아",
    "금전적지원",
    "사비로",
    "업체로부터",
    "협찬",
    "제품또는서비스를",
    "업체에서",
    "소정의",
    "고료를",
    "할인을",
    "레뷰",
    "강남오빠",
    "디너의여왕",
    "서울오빠",
    "이놀자데뷰"
]

In [3]:
def load_img(url: str):
    res = requests.get(url)
    img = Image.open(BytesIO(res.content))
    
    return img

In [4]:
HAN_REGEX = re.compile('[^가-힣]')

In [5]:
files = glob('./data/semi_struct_ocr/*.json')
files

['./data/semi_struct_ocr/홍대 회식 맛집.json',
 './data/semi_struct_ocr/홍대 데이트 맛집.json',
 './data/semi_struct_ocr/성수 데이트 맛집.json',
 './data/semi_struct_ocr/압구정 회식 맛집.json',
 './data/semi_struct_ocr/강남역 데이트 맛집.json',
 './data/semi_struct_ocr/영등포 회식 맛집.json',
 './data/semi_struct_ocr/성수 회식 맛집.json',
 './data/semi_struct_ocr/압구정 데이트 맛집.json',
 './data/semi_struct_ocr/강남역 회식 맛집.json',
 './data/semi_struct_ocr/범계 회식 맛집.json',
 './data/semi_struct_ocr/범계 데이트 맛집.json',
 './data/semi_struct_ocr/영등포 데이트 맛집.json']

In [6]:
with open(files[0], 'r', encoding='utf-8') as f:
    blog = json.load(f)

In [7]:
blog[141]

{'blog_name': '잠깐의 여유 log',
 'date': '20230414',
 'url': 'https://blog.naver.com/emma8072/223074462563',
 'title': "<b>홍대</b> 삼겹살 <b>맛집</b> '청돈옥' 합정 <b>회식</b>장소 구워주는 고기집",
 'contents': [{'no': 1,
   'info': 'text',
   'content': '한양도성길을 걸은 뒤에 홍대로 이동했다.\n반나절을 걸은 덕에 체력이 많이 소진되어 저녁식사로  고기 섭취를 하기로 했다.\n홍대, 합정역 삼겹살이 맛있다는 청돈옥으로 방문!'},
  {'no': 2,
   'info': 'img',
   'content': 'https://postfiles.pstatic.net/MjAyMzA0MTFfMTA3/MDAxNjgxMTg4Njk2NTc3.R5h3FduW6eYRns7CbjAZNYS6zztHUlKiVMjq7aS9gHMg.8ufRQtB5K6hILJxRfPIkZS3b3H-lTMdZRsj4NrZ5fvUg.JPEG.emma8072/SE-e74f5226-d73f-11ed-aea7-09c59096d87f.jpg?type=w966',
   'ocr_text': ''},
  {'no': 3,
   'info': 'text',
   'content': '홍대 회식 삼겹살 맛집\n- 청돈옥 -\n#홍대삼겹살 #홍대고기집 #합정삼겹살 #합정회식'},
  {'no': 4,
   'info': 'img',
   'content': 'https://postfiles.pstatic.net/MjAyMzA0MTFfMjk1/MDAxNjgxMTg3OTAzOTk1.KfyAIrS7GtOyOc_Cl1Cq96WSoUCGQkMiVToP65CWbxsg.xV1tqA1ZYgKd9V6Zhlt2hN9r4JqSl7qHljn0vOgOZsgg.JPEG.emma8072/SE-e70d3ffd-d73f-11ed-aea7-2b8da295a1f0.jpg?type=w966',
   '

In [8]:
df = []
for article in blog:
    easy_text = ""
    tess_text = ""
    plain_text = ""
    for p in article['contents']:
        if p['info'] in ['img', 'gif']:
        
            if p.get('tess_text'):
                easy_text += ' ' + p['easy_text']
                tess_text += ' ' + p["tess_text"]
        elif p['info'] == 'text':
            plain_text += ' ' + p['content']
            
    df.append({
        "article_id": article["id"],
        "blog_url": article['url'],
        "plain_text": HAN_REGEX.sub('', plain_text),
        "easyocr": HAN_REGEX.sub('', easy_text),
        "tesseractocr": HAN_REGEX.sub('', tess_text),
        "plain_origin": plain_text,
        "easy_origin": easy_text,
        "tesseract_origin": tess_text
    })
df = pd.DataFrame(df)

In [9]:
for kywd in ad_keywords:
    df.loc[df[df.plain_text.str.contains(kywd)].index, 'label'] = 1
    df.loc[df[df.tesseractocr.str.contains(kywd)].index, 'label'] = 1
    df.loc[df[df.easyocr.str.contains(kywd)].index, 'label'] = 1
df['label'] = df.label.fillna(0)

In [10]:
df[df.label == 0]['easy_origin'].apply(lambda x: x[-30:]).loc[50:100]

51     
52     
53     
54     
55     
57     
58     
59     
60     
61     
62     
63     
64     
65     
66     
67     
68     
69     
70     
71     
72     
73     
74     
76     
77     
79     
82     
83     
84     
86     
89     
90     
91     
92     
93     
94     
95     
97     
98     
99     
100    
Name: easy_origin, dtype: object

In [50]:
df.loc[74]

article_id                                     oursophie.223210064823
blog_url                https://blog.naver.com/oursophie/223210064823
plain_text          일관된분위기와정돈된스타일이마음에들어서한번이상재방문할것같은홍대맛집회식후에가기좋은터줏대...
easyocr             안녕어세그벼각테일웬손정보씨분접인기메뉴소주맥주생맥주양주외인각테일취향대로슬길수있는다양한...
tesseractocr        개개이아으이터으으려취향대로즐길수있는다양한술과이므이더미그오버리기으으터년일본식오절임트러...
plain_origin         일관된 분위기와 정돈된 스타일이 마음에 들어서\n한 번 이상 재방문할 것 같은 홍...
easy_origin          안녕어세 SEVEN PUB SEVEN 그#7 EM PUB 벼:? B 각테일 웬손정...
tesseract_origin     
  \n
 
  \n         \n\n1\n\n개 개 이아 으이터 으으려\...
label                                                             1.0
Name: 74, dtype: object

In [51]:
df[['tesseractocr', 'easy_origin']]

,tesseractocr,easy_origin
0,벌집업데기막창사이트맥주때심술도도청하이캐삐때맛깔스러운잔반찬본포스팅은업체로부터서비스만을,dNI LaVIo MRAO 장안집 VAENG BAN JIB 8292 OPEMI12...
1,범는는트는는가자허오므수으로이이이오으로오고아노겨분그오그오이개이아제도이기으으에오미쿄록대...,"ymm로 IGe! drvA, T - Yeonnam Iohil VMiaire Cc..."
2,,K 'AI rao 9 @l aISL 'ORIGINAL A
3,는노털로오느고기류대표메뉴사구시그니쳐고을수스대오개사이드벡씨네색후토마끼나으윤씨네녕면물비...,올 호 석 듣데이OSBS ' 연막단 매스럽이 극찬한 생방송 연막람 트데이OSR...
4,브쁘으설명도친절하게셔서취향에맞게잘특별한날부모님이도너무너무좋은한우특별고깃집문할게요메뉴...,팔로우 여기논 회사 회식할다 자주오는데 올 더가다 만족하고 갑니다 등심은 들튼이...
...,...,...
883,므노고그이배게나사오나테그나나노녀누너벼저애애자고스애스나원나사내어원자원초계비빔면이글은레...,@a Tu로숲 81u0101989 돈 욕 m문욕 기본에 충실이켜습니다 우리움...
884,,
885,애드온밸기투구거홍대오잉크파는반합으로고기와할께세공되는파는빈함뚜경이익허주셔요고거는자주고...,"구이 흉대오임크 ##7빵 'Ilit"" Ii""st kAy eal I:l.din (A..."
886,비무쁜매드글대나그개고으니미는을다아그뷰더다더너결호주개블배노가구가오오내뼈오거도이개백개매...,Hello Owi( 8 흙 통 피 { 푹 틀 '290031 { 흉#- OuYMTm...


In [54]:
df[df.label == 1]['easy_origin'][0]

' dNI LaVIo MRAO 장안집 VAENG BAN JIB 8292 OPEMI12:0O PM CLOSE 02.00 AM WIFI NOPET TOILET DELIVERY NO SMOKING GCTV     1Ua 쟁반집 돼지고기 \'활삼 *99 j119 n심 "109 군 eL9 파심  119 1수부위 \'119 소고기 우상적 *30 삼기심 140 감비살 \'HO 모툼 정반 우궁반 r) 32302 우산경 + 기비살 + 실치실 5V 튼장반 a 3302 심중+ 핵살 + 항경 + 옥수부히 녕경반 ro) 30 8심+8화심 + 우상려 (터1다 회 식 세트 온정반 d + 우정반 o) + v식다밥구이 \'910\' 석배 벌집접데기 막창 생반소시지 \'수제숙불닭발 "89 *120 "69 #10.0 사 이드 된장말이밥 김치말이밥 된장찌개 김치찌개 60 "60 "5.0 #50 꽃게라면 냉면 물 비) 계란짐 #60 "5.0 "5.0 생반부음밥 지즈쟁반볶음밥 "79 "99 음료 / 주류 소주 "50 맥주 #50 나Arone_n 낙벼 o어e_ 심술7도 12도 "60 청하 "50 계43n/M 음료수  표하다 사이다 터개 #20 NEIII 맛충사 사청제 " @활 우정반 무무 신@d9) 돈쟁반 BEEF :PoRX SET -   d무 1688-2922 에이드 강추! 2"\' YC "n \'양 계절마다 맛깔스러운 제절반찬이 기본서비스 다양한반찬 \'? 수:191 SET - { 흙 4분이기다림 아이템 물 총 :          76 {                      *본 포스팅은 업체로부터 서비스만을 제공받아 솔직하게 작성 포스팅 입니다*'